## Pyro General Workflow

In this notebook, I am taking the time to understand the basic workflow and 
syntax for the PPL Pyro.

In Bayesian analysis, theory usually begins with the following triple:
(data, model, prior). From this, we compute a posterior which is then
used for further inference tasks. My question becomes: how does Pyro take this
data-model-prior story and perform tasks?

### Pyro has two workflows one can use to go from model to posterior:

#### 1. MCMC Workflow
1. We define the generative model $p(w, \mathbf{y}) = p(\mathbf{y}|w)\,p(w)$
using a python function ```def model(data)``` where inside we define:
    1. The prior, 
    2. The likelihood.
2. Set up MCMC sampling:
    * MCMC kernel -> HMC, NUTS, etc.
    * MCMC driver -> takes user arguements (e.g. number of samples) and performs
    MCMC sampling once called to run.
    * Extract samples from MCMC driver.


#### 2. Variational Inference Workflow
TO DO.

### The Key Pyro Primitives

1. ``` pyro.sample(name, dist, obs=None) ```

    Prior or likelihood draws. 
    * ```obs=0``` samples prior;
    * ```obs=tensor``` samples likelihood.
    

2. ``` pyro.param(name, init_tensor, constraint=...) ``` 

    Deterministic learnable parameters

### Practice:
Let us put all of this into practice on a basic Beta-Bernoulli coin-flip model
performing both workflows: MCMC and Variational Inference using Pyro.

In [ ]:
import torch
import pyro
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS, SVI, Trace_ELBO
import pyro.optim as optim

# Reset internal parameter state 
# (important to reset params when re-nunning notebooks)
pyro.clear_param_store()

## Generate coin-flip data
# True parameter w
true_w = 0.7
N = 50
# length-N tensor of 0's and 1's indicating our observed flips
data = dist.Bernoulli(torch.tensor(true_w)).sample((N,)) # Shape (N,)

## Prior and likelihood
# Define the generative model p(w, y)
def model(data):
    # prior on w ~ Beta(alpha0, beta0)
    alpha0 = torch.tensor(1.0) 
    beta0 = torch.tensor(1.0)
    # "w" is the latent variable name used later to collect posterior samples
    w = pyro.sample("w", dist.Beta(alpha0, beta0))

    # Likelihood for each observed flip
    # pyro.plate declares that the random choices inside are conditionally
    # independent and identically distributed over the "data" plate of size N
    with pyro.plate("data", len(data)):
        # Define y_i ~ Bernoulli(w) as the likelihood
        # obs=data tells Pyro to condition on the observed data instead of
        # sampling new ones. If sampling new ones, we'd get the posterior over w
        pyro.sample("obs", dist.Bernoulli(w), obs=data)

## MCMC (using NUTS)
# Set up a Hamiltonian Monte Carlo kernel using our model as the target
nuts_kernel = NUTS(model)
# Set up the MCMC driver with num_samples to keep and warmup_steps as burn-in
mcmc = MCMC(nuts_kernel, num_samples=1000, warmup_steps=500)
# Actually runs the markov chain 
mcmc.run(data)
# Returns a dictionary of tensors keyed by sample site name
# For us, we have key: "w", value: tensor of shape (num_samples, )
posterior_samples_MCMC = mcmc.get_samples()

# Extract the samples drawn from the dictionary with key "w"
w_samples_MCMC = posterior_samples_MCMC["w"]
# Posterior summary statistics (mean and 90% CI)
w_mean_MCMC = w_samples_MCMC.mean()
w_CI_MCMC = w_samples_MCMC.quantile(torch.tensor([0.05, 0.95]))

print(f"MCMC Output for w:\nMEAN: {w_mean_MCMC},\n90% CI: {w_CI_MCMC}.")

## Variational Inference (using SVI)
# TO DO

Sample: 100%|██████████| 1500/1500 [00:05, 268.22it/s, step size=9.46e-01, acc. prob=0.910]


MCMC Output for w:
MEAN: 0.6509367823600769,
90% CI: tensor([0.5410, 0.7570]).


In [10]:
w_mean_MCMC
w_samples_MCMC.mode()

torch.return_types.mode(
values=tensor(0.7360),
indices=tensor(574))